It might be useful to run 'pip3 install &lt;library&gt; --upgrade" to make sure you have latest versions of pandas, matplotlib, etc.

In [1]:
import gzip            
import io              
import numpy as np
import pandas as pd    
import pymysql.cursors 
import rdflib
from rdflib import Namespace
import urllib.request  

import matplotlib.pyplot as plt
plt.style.use('ggplot')

%matplotlib inline

In [2]:
# Connect to the database
# I will email you the file sebastia_adsq_params.txt . It's just a four line text file.
# Put that in directory and the following line will read it.
db_params = open("sebastia_adsqro_params.txt").read().split()

In [3]:
# Connect to the database

connection = pymysql.connect(host=db_params[0],
                             user=db_params[1],
                             password=db_params[2],
                             db=db_params[3],
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

## Turn 'ramphs' into Triples

In [4]:
sql = "select * from ramphs"

with connection.cursor() as cursor:

    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()

df = pd.DataFrame(result, columns = names)
df.head()

,id,title,label,pleiades,welch,golvin,type,capacity,modcountry,province,...,certainty,youtube,extmajor,extminor,arenamajor,arenaminor,latitude,longitude,elevation,pleiades_id
0,duraEuroposAmphitheater,Amphitheater at Dura Europos,Dura,http://pleiades.stoa.org/places/893989,,129,amphitheater,1000.0,Syria,syria,...,,,50.0,44.0,31.0,25.0,34.74985500000,40.7289260000,223,893989
1,arlesAmphitheater,Amphitheater at Arles,Arles,http://pleiades.stoa.org/places/148217,,154,amphitheater,20000.0,France,narbonensis,...,,https://www.youtube.com/watch?v=oCz-76hb1LU,136.0,107.0,47.0,32.0,43.67777800000,4.6311110000,21,148217
2,lyonAmphitheater,Amphitheater at Lyon,Lyon,http://pleiades.stoa.org/places/167717,,,amphitheater,20000.0,France,lugdunensis,...,,,105.0,NaN,67.6,42.0,45.77055600000,4.8305560000,206,167717
3,ludusMagnusArena,Ludus Magnus Arena,Ludus Magnus,http://pleiades.stoa.org/places/423025,,,practice-arena,NaN,Italy,,...,,,NaN,NaN,NaN,NaN,41.88995000000,12.4949130000,22,423025
4,romeFlavianAmphitheater,Flavian Amphitheater at Rome,Colosseum,http://pleiades.stoa.org/places/423025,,152,amphitheater,50000.0,Italy,,...,,https://www.youtube.com/watch?v=kxwenPo1grc,189.0,156.0,83.0,48.0,41.89016900000,12.4922690000,22,423025


In [5]:
pseudo_triples = pd.melt(df,id_vars='id').sort_values('id')
pseudo_triples.head()

,id,variable,value
3769,agenAmphitheater,extminor,NaN
4805,agenAmphitheater,longitude,0.6221220000
1956,agenAmphitheater,modcountry,France
2992,agenAmphitheater,certainty,
661,agenAmphitheater,pleiades,http://pleiades.stoa.org/places/138169


In [6]:
for t in pseudo_triples.iterrows():
    s = 'ex:%s' % t[1]['id']
    p = 'ex:%s' % t[1]['variable']
    if t[1]['variable'] in ['arenamajor','arenaminor','capacity','elevation','extmajor','extminor','longitude','latitude']:
        print('%s %s %s' % (s,p,t[1]['value']))
    else:
        print('%s %s "%s"' % (s,p,t[1]['value']))

ex:agenAmphitheater ex:extminor nan
ex:agenAmphitheater ex:longitude 0.6221220000
ex:agenAmphitheater ex:modcountry "France"
ex:agenAmphitheater ex:certainty ""
ex:agenAmphitheater ex:pleiades "http://pleiades.stoa.org/places/138169"
ex:agenAmphitheater ex:type "amphitheater"
ex:agenAmphitheater ex:label "Agen"
ex:agenAmphitheater ex:welch ""
ex:agenAmphitheater ex:title "Amphitheater at Agen"
ex:agenAmphitheater ex:province "aquitania"
ex:agenAmphitheater ex:latitude 44.20654500000
ex:agenAmphitheater ex:pleiades_id "138169"
ex:agenAmphitheater ex:chronogrp "Imperial"
ex:agenAmphitheater ex:italyregion ""
ex:agenAmphitheater ex:golvin ""
ex:agenAmphitheater ex:extmajor nan
ex:agenAmphitheater ex:elevation 52
ex:agenAmphitheater ex:arenamajor nan
ex:agenAmphitheater ex:capacity 15000.0
ex:agenAmphitheater ex:arenaminor nan
ex:agenAmphitheater ex:youtube ""
ex:ainHedjahAmphitheater ex:extmajor 57.0
ex:ainHedjahAmphitheater ex:pleiades "http://pleiades.stoa.org/places/314857"
ex:ainHedja

In [7]:
g = rdflib.Graph()
RDF = Namespace("http://example.org/people/")
EXid = Namespace("http://example.org/id/")
EXvocab = Namespace("http://example.org/vocab/")

In [8]:


pseudo_triples = pd.melt(df,id_vars='id').sort_values('id')

for t in pseudo_triples.iterrows():
    s = rdflib.URIRef('http://example.org/id/%s' % t[1]['id'])
    p = rdflib.URIRef('http://example.org/vocab/%s' % t[1]['variable'])

    g.add((s,rdflib.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),EXvocab.amphitheater))
    if t[1]['variable'] in ['arenamajor','arenaminor','capacity','elevation','extmajor','extminor','longitude','latitude']:
        o = rdflib.Literal(float(t[1]['value']))
        g.add((s,p,o))
    elif t[1]['variable'] in ['pleiades']:
        o = rdflib.URIRef(t[1]['value'])
        g.add((s,p,o))
    else:
        o = rdflib.Literal(str(t[1]['value']))
        g.add((s,p,o))

In [9]:
result = g.query(
        """SELECT * WHERE {
            ?s ?p ?o .
                } LIMIT 10""")
for r in result:
    print(r.s)

http://example.org/id/gemellaeAmphitheater
http://example.org/id/aquincumMilitaryAmphitheater
http://example.org/id/megiddoAmphitheater
http://example.org/id/sanGiovanniAmphitheater
http://example.org/id/luccaAmphitheater
http://example.org/id/anazarbusAmphitheater
http://example.org/id/arnsburgAmphitheater
http://example.org/id/chietiAmphitheater
http://example.org/id/mactarisAmphitheater
http://example.org/id/sbeitlaAmphitheater


In [10]:
# or
pd.DataFrame(result.bindings)

,o,p,s
0,http://example.org/vocab/amphitheater,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.org/id/gemellaeAmphitheater
1,19.03906599,http://example.org/vocab/longitude,http://example.org/id/aquincumMilitaryAmphithe...
2,http://example.org/vocab/amphitheater,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.org/id/megiddoAmphitheater
3,amphitheater,http://example.org/vocab/type,http://example.org/id/sanGiovanniAmphitheater
4,,http://example.org/vocab/welch,http://example.org/id/luccaAmphitheater
5,Amphitheater at Anazarbus,http://example.org/vocab/title,http://example.org/id/anazarbusAmphitheater
6,Arnsburg,http://example.org/vocab/label,http://example.org/id/arnsburgAmphitheater
7,413333,http://example.org/vocab/pleiades_id,http://example.org/id/chietiAmphitheater
8,119,http://example.org/vocab/golvin,http://example.org/id/mactarisAmphitheater
9,60.0,http://example.org/vocab/extminor,http://example.org/id/sbeitlaAmphitheater


## Turn 'chronogroups' into Triples

In [11]:
sql = "select * from ramphs_chronogrps"

with connection.cursor() as cursor:

    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()

df = pd.DataFrame(result, columns = names)
df.head()

,id,start,end
0,Republican,-70,-31
1,Caesarean,-49,-44
2,Augustan,-27,14
3,Julio-Claudian,-27,68
4,Neronian,54,68


In [12]:
df.rename(index=str, columns={"id": "label"}, inplace = True)
newcol = df['label'].replace(r' ', '', regex = True)
df = df.assign(idd = newcol)
df.rename(index=str, columns={"idd": "id"}, inplace = True)
df

,label,start,end,id
0,Republican,-70,-31,Republican
1,Caesarean,-49,-44,Caesarean
2,Augustan,-27,14,Augustan
3,Julio-Claudian,-27,68,Julio-Claudian
4,Neronian,54,68,Neronian
5,Flavian,69,96,Flavian
6,First Century,1,99,FirstCentury
7,Late1stEarly2nd,75,125,Late1stEarly2nd
8,Hadrianic,117,138,Hadrianic
9,Second Century,100,199,SecondCentury


In [13]:
pseudo_triples = pd.melt(df,id_vars='id').sort_values('id')
pseudo_triples.head()

,id,variable,value
34,Augustan,end,14
2,Augustan,label,Augustan
18,Augustan,start,-27
1,Caesarean,label,Caesarean
17,Caesarean,start,-49


In [14]:
pseudo_triples = pd.melt(df,id_vars='id').sort_values('id')
pseudo_triples
for t in pseudo_triples.iterrows():
    s = rdflib.URIRef('http://example.org/id/%s' % t[1]['id'])
    p = rdflib.URIRef('http://example.org/vocab/%s' % t[1]['variable'])
    
    g.add((s,rdflib.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),EXvocab.chronogrp))
    if t[1]['variable'] in ['start','end']:
        o = rdflib.Literal(float(t[1]['value']))
        g.add((s,p,o))
    elif t[1]['variable'] in ['']:
        o = rdflib.URIRef(t[1]['value'])
        g.add((s,p,o))
    else:
        o = rdflib.Literal(str(t[1]['value']))
        g.add((s,p,o))

In [15]:
result = g.query(
        """SELECT ?p ?o WHERE {
            <http://example.org/id/FirstCentury> ?p ?o .
                } LIMIT 10""")
for r in result:
    print(r.o)

http://example.org/vocab/chronogrp
99.0
First Century
1.0


In [16]:
# or
pd.DataFrame(result.bindings)

,o,p
0,http://example.org/vocab/chronogrp,http://www.w3.org/1999/02/22-rdf-syntax-ns#type
1,99.0,http://example.org/vocab/end
2,First Century,http://example.org/vocab/label
3,1.0,http://example.org/vocab/start


In [17]:
result = g.query(
        """SELECT * WHERE {
            ?ramph a <http://example.org/vocab/amphitheater> ;
                    <http://example.org/vocab/chronogrp> ?rchrono .
                    
            ?chronogrp <http://example.org/vocab/label> ?rchrono ;
                       <http://example.org/vocab/end> ?end
                } ORDER BY ?end""")

pd.DataFrame(result.bindings)

,chronogrp,end,ramph,rchrono
0,http://example.org/id/Caesarean,-44.0,http://example.org/id/espejoAmphitheater,Caesarean
1,http://example.org/id/Caesarean,-44.0,http://example.org/id/carmonaAmphitheater,Caesarean
2,http://example.org/id/Caesarean,-44.0,http://example.org/id/corinthAmphitheater,Caesarean
3,http://example.org/id/Republican,-31.0,http://example.org/id/liternoAmphitheater,Republican
4,http://example.org/id/Republican,-31.0,http://example.org/id/suessaAruncaAmphitheater,Republican
5,http://example.org/id/Republican,-31.0,http://example.org/id/avellaAmphitheater,Republican
6,http://example.org/id/Republican,-31.0,http://example.org/id/capuaRepublicanAmphitheater,Republican
7,http://example.org/id/Republican,-31.0,http://example.org/id/antiochAmphitheater,Republican
8,http://example.org/id/Republican,-31.0,http://example.org/id/noleAmphitheater,Republican
9,http://example.org/id/Republican,-31.0,http://example.org/id/sutriumAmphitheater,Republican
